In [ ]:
# dependencies
import numpy as np
from scipy import interpolate
from scipy import stats
import healpy as hp  # NOTE: you can neglect warning about version mismatch in CFITSIO if necessary 

N_side = 2048      # Both maps will have this N_side. Healpix map has 12 * N_side**2 pixels

from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy import units as u

import pandas as pd

import emcee

import matplotlib.pyplot as plt
import os, sys, shutil
import random
import sklearn.metrics as slm

import corner
from scipy.interpolate import interp1d
%matplotlib inline

pkdir = "/pscratch/sd/s/sbrisin/boss_cib_cmass/cibcmass/hmvec"
sys.path.insert(0, pkdir)
# Import hmvec as hm
from hmvec import hmvec as hm

This part of the notebook is for prepping the data, getting the stuff calculated

In [ ]:
# load maps 
CIB_map = hp.read_map('/pscratch/sd/s/sbrisin/boss_cib_cmass/cibcmass/notebooks/COM_CompMap_CIB-GNILC-F545_2048_R2.00(2).fits')        # read CIB map
CIB_mask = hp.read_map('/pscratch/sd/s/sbrisin/boss_cib_cmass/catalogues/cib_mask.fits')                                    # read CIB mask

CIB_masked = hp.ma(CIB_map)
CIB_masked.mask = np.logical_not(CIB_mask)

In [ ]:
ef read_nobs_pyfits(filename):
    with fits.open(filename, memmap=True) as hdul:
        data = (hdul[1].data)
        return np.shape(data)[0], hdul[1].columns.names

def read_test_pyfits(filename, colname):
    with fits.open(filename, memmap=True) as hdul:
        data = (hdul[1].data[colname])
        return data.copy()

def get_BOSS_data(gal):
    nObs, cols = read_nobs_pyfits(gal)
    colnames = [x for x in cols if x in ['ID', 'RA', 'DEC', 'Z', 'NZ', 'BOSS_SPECOBJ_ID',
                                         'BOSS_TARGET1', 'BOSS_TARGET2', 'EBOSS_TARGET0', 'ZOFFSET', 'TARGETOBJID',
                                         'OBJID', 'PLUG_RA', 'PLUG_DEC', 'Z']]
    ncols = len(colnames)
    myGalaxy = pd.DataFrame(data=np.zeros([nObs, ncols]), columns=colnames)
    for rowname in myGalaxy.columns:
        myGalaxy[rowname] = read_test_pyfits(gal, rowname).byteswap().newbyteorder()
    print(myGalaxy.columns)
    myGalaxy = myGalaxy.sort_values(by=['Z'])
    return myGalaxy




boss_catalog_path_s = '/pscratch/sd/s/sbrisin/boss_cib_cmass/cibcmass/notebooks/galaxy_DR12v5_CMASS_South (1).fits'
boss_catalog_path_n = '/pscratch/sd/s/sbrisin/boss_cib_cmass/cibcmass/notebooks/galaxy_DR12v5_CMASS_North (1)(1).fits'
# Load the relevant entries of the catalog (we will be mostly concerned with RA, DEC, and redshift)

boss_catalog_s = get_BOSS_data(boss_catalog_path_s)
boss_catalog_n = get_BOSS_data(boss_catalog_path_n)

datas = get_BOSS_data(boss_catalog_path_s)
datan = get_BOSS_data(boss_catalog_path_n)
datas = pd.DataFrame(datas)
datasmodd = pd.DataFrame(datas[(datas.Z > 0.4) * (datas.Z < .5)])
datanmodd = pd.DataFrame(datan[(datan.Z > 0.4) * (datan.Z < .5)])

modtot = pd.concat([datanmodd,datasmodd])


In [ ]:
nside = 2048
npix = hp.nside2npix(nside)
c_icrs_s = SkyCoord(ra=modtot['RA']*u.degree, dec=modtot['DEC']*u.degree, frame='icrs')
indices = hp.ang2pix(nside, c_icrs_s.galactic.l.value, c_icrs_s.galactic.b.value, lonlat=True)
gal_hpmap = np.zeros(npix, dtype=np.float)
np.add.at(gal_hpmap, indices, 1)

def get_dn_from_map(gal_hpmap):
    ''' Convert a galaxy number map obtained using get_map_from_catalog() to one of fractional
        overdensity. Return also the mask
    ''' 
    #Total number counts. Should convert to delta_n/n. Make a mask first!
    gal_downgraded = hp.ud_grade(gal_hpmap, 64)
    gal_mask = np.zeros(len(gal_downgraded))
    gal_mask[(gal_downgraded > 0)] = 1
    nside = hp.npix2nside(len(gal_hpmap))
    gal_mask = hp.ud_grade(gal_mask, nside) # bring it back to the original resolution
    
    # Convert to delta_n / n (fractional fluctuations: this is what we can predict from theory)
    mean_gal = np.sum(gal_hpmap * gal_mask) / np.sum(gal_mask)
    gal_masked_dn = (gal_hpmap * gal_mask) / mean_gal - 1.
    gal_masked_dn = gal_mask * gal_masked_dn
    return gal_masked_dn, gal_mask


gal_dn, gal_mask = get_dn_from_map(gal_hpmap)
gal_dn.max()

In [ ]:
# CMASS map, total number counts. Should convert to delta_n/n. Make a mask first!
CMASS_map = gal_hpmap # mask should be made from regular density 
CMASS_downgraded = hp.ud_grade(CMASS_map, 64)
CMASS_mask = np.zeros(len(CMASS_downgraded))
CMASS_mask[(CMASS_downgraded > 0)] = 1
CMASS_mask = hp.ud_grade(CMASS_mask, N_side) # bring it back to the original resolution
coord= ['G','C']
# Convert to a masked array for better plotting
CMASS_masked_dn = hp.ma(gal_dn)
CMASS_masked_dn.mask = np.logical_not(CMASS_mask)

In [ ]:
# The "total" mask is product of PLANCK and CMASS masks
mask_tot = CMASS_mask * CIB_mask
fsky = np.sum(mask_tot) / len(mask_tot) # fsky for the cross-correlation
fsky_CIB = np.sum(CIB_mask) / len(CIB_mask)
fsky_CMASS = np.sum(CMASS_mask) / len(CMASS_mask)
print("f_sky cross = " + str(fsky))
print("f_sky CIB = " + str(fsky_CIB))
print("f_sky CMASS = " + str(fsky_CMASS))

In [ ]:
Cl_cross = hp.anafast(CIB_masked, map2 = CMASS_masked_dn, lmax = 2000)
ls = np.arange(len(Cl_cross))
pixwinf = hp.pixwin(N_side)[0:len(Cl_cross)]
# further divide by one power of beam
Cl_cross = Cl_cross / (pixwinf **2)                # Remove pixel window function
Cl_cross = Cl_cross / fsky                         # Undo the dilution caused by having observed only part of the sky

In [ ]:
# Same as before!
Cl_CIB = hp.anafast(CIB_masked, lmax = 2000)
Cl_CMASS = hp.anafast(CMASS_masked_dn, lmax = 2000)
Cl_CIB = Cl_CIB / (pixwinf **2)
Cl_CMASS = Cl_CMASS / (pixwinf **2)
Cl_CIB = Cl_CIB / fsky_CIB
Cl_CMASS = Cl_CMASS / fsky_CMASS


this is all the stuff for making the actual covariance matrix

In [ ]:
def cov_mat_nocib(cii,cgg,cig,l):
    f_sky_cross = 0.249634150739921
    f_sky_CIB = 0.5787552197774252
    f_sky_CMASS = 0.2552083333333333

    # cgg, cll, and cig are going to be the valeus pre bin?
    var_gg = (2*cgg)/((2*l+1)*(f_sky_CMASS))
    var_ig = (cgg*cii+cig**2)/((2*l+1)*f_sky_cross)
    cov_cgg_cgi = (2*cgg*cig)/((2*l+1)*f_sky_CMASS)
    a = np.array([var_gg,cov_cgg_cgi])
    b = np.array([cov_cgg_cgi,var_ig])
    return l,var_gg,var_ig,cov_cgg_cgi

In [ ]:
ells = np.arange(300,1000)
b = []
for l in ells:
    clcib_l = Cl_CIB[l]
    clcmass_l = Cl_CMASS[l]
    clcross_l = Cl_cross[l]
    y = cov_mat_nocib(clcib_l,clcmass_l,clcross_l,l)
    b.append(y)
    
def bins(n_bin,min_l,max_l): # calculate the bins size
    x = []
    b = 0
    size = (max_l-min_l)/n_bin
    while min_l <= 1000:
        x.append(min_l)
        min_l = min_l + size
        print(min_l)
    return np.array(x)

bins(5,300,1000)

In [1]:
# add the stuff from the meeting yesterday

this is for the mcmc!! THIS IS ALL FOR .4 to .5 Z bins!!!!!!

In [ ]:
# set up all the HMVEC stuff

ells = np.arange(3000)
cib_freq = 545 * 1e9


zs = np.linspace(.4, .5, 10)
ms = np.geomspace(1e10, 1e17, 20) # revisit the webb sky, 
#lower mass limit and see if the disagreement goes away 
ks = np.geomspace(1e-4, 100, 100)

hcos = hm.HaloModel(zs, ks, ms=ms)
hcos.set_cibParams('vierro')

        # a and b are the z range
hcos.cib_params['alpha'] = 0.2
hcos.cib_params['beta'] = 1.6
hcos.cib_params['gamma'] = 1.7  # not in Viero, so using Planck13
hcos.cib_params['delta'] = 2.4
hcos.cib_params['Td_o'] = 20.7
logmeff = hcos.cib_params['logM_eff']
hcos.cib_params['var'] = 0.3
l0 = hcos.cib_params['L_o']
hcos.add_hod(name="CMASS", mthresh=10**12 + zs*0.)
ells = np.arange(300,1000)

In [ ]:
def cgg_mod(theta):
    x,y = theta
    gdndz = np.array([3091.0, 4697.0,7307.0,11655.0,18705.0,25476.0,	31238.0,37006.0	,40371.0,	43370.0])
    zs = np.array([0.40000978,0.41000876,0.42000774,0.4300067,0.4400057,0.45000467,0.46000364,0.47000262,0.4800016,0.49000058,0.4999995529651642])
    hcos.cib_params['L_o'] = x
    hcos.cib_params['logM_eff'] = y
            # Power spectra for CIB x galaxies
    Pgg_1h = hcos.get_power_1halo('CMASS', 'CMASS', nu_obs=np.array([cib_freq]))
    Pgg_2h = hcos.get_power_2halo('CMASS', 'CMASS', nu_obs=np.array([cib_freq]))

    Cl_gg_1h = hcos.C_gg(ells, hcos.zs, hcos.ks, Pgg_1h, gzs=zs, gdndz=gdndz)
    Cl_gg_2h = hcos.C_gg(ells, hcos.zs, hcos.ks, Pgg_2h, gzs=zs, gdndz=gdndz)
    
    # need to end up with one vector thats c_gi for each l bin, c_gg for each l bin
    tot = Cl_gg_1h + Cl_gg_2h
    
    b1 = tot[ells == 70]
    b2 = tot[ells == 210]
    b3 = tot[ells == 350] 
    b4 = tot[ells == 490.0]
    b5 = tot[ells == 630]
    b = np.array([b1,b2,b3,b4,b5])
    return b

def cii_mod(theta):
    x,y = theta
    zs = np.linspace(.01,5,20)
    ms = np.geomspace(1e10,1e17,200)
    ks = np.geomspace(1e-4,100,1001)
    hcos = hm.HaloModel(zs,ks,ms=ms)
    hcos.set_cibParams('vierro')
    hcos.cib_params['L_o'] = x
    hcos.cib_params['logM_eff'] = y
            # Power spectra for CIB x galaxies
    Pii_1h = hcos.get_power_1halo('cib', 'cib', nu_obs=np.array([cib_freq]))
    Pii_2h = hcos.get_power_2halo('cib', 'cib', nu_obs=np.array([cib_freq]))

    Cl_ii_1h = hcos.C_ii(ells, hcos.zs, hcos.ks, Pii_1h)
    Cl_ii_2h = hcos.C_ii(ells, hcos.zs, hcos.ks, Pii_2h)
    
    # need to end up with one vector thats c_gi for each l bin, c_gg for each l bin
    tot = Cl_ii_1h + Cl_ii_2h
    
    b1 = tot[ells == 70]
    b2 = tot[ells == 210]
    b3 = tot[ells == 350] 
    b4 = tot[ells == 490.0]
    b5 = tot[ells == 630]
    b = np.array([b1,b2,b3,b4,b5])
    return b

def cig_mod(theta):
    x,y = theta
    gdndz = np.array([3091.0, 4697.0,7307.0,11655.0,18705.0,25476.0,31238.0,37006.0	,40371.0,43370.0])
    gzs = np.array([0.40000978,0.41000876,0.42000774,0.4300067,0.4400057,0.45000467,0.46000364,0.47000262,0.4800016,0.49000058])
    new_gdndz = np.interp(hcos.zs, gzs, gdndz, left=0, right=0)
    hcos.cib_params['L_o'] = x
    hcos.cib_params['logM_eff'] = y
            # Power spectra for CIB x galaxies
    PgI_1h = hcos.get_power_1halo('CMASS', 'cib', nu_obs=np.array([cib_freq]))
    PgI_2h = hcos.get_power_2halo('CMASS', 'cib', nu_obs=np.array([cib_freq]))

    Cl_gI_1h = hcos.C_gI(ells, hcos.zs, hcos.ks, PgI_1h, gzs=gzs, gdndz= new_gdndz)
    Cl_gI_2h = hcos.C_gI(ells, hcos.zs, hcos.ks, PgI_2h, gzs=gzs, gdndz= new_gdndz)
    
    # need to end up with one vector thats c_gi for each l bin, c_gg for each l bin
    tot = Cl_gI_1h + Cl_gI_2h
    
    df = pd.DataFrame({'ells': ells, 'tot': tot})
    b1 = float(tot[ells == 370.0])
    b2 = float(tot[ells == 510.0])
    b3 = float(tot[ells == 650.0]) 
    b4 = float(tot[ells == 790.0])
    b5 = float(tot[ells == 930.0])
    b = np.array([b1,b2,b3,b4,b5])
    return b

In [ ]:
def model(theta, dat = dat):
    x,y =  theta
    return (cig_mod(theta))
def lnlike(theta,dat,cov,cinv):
    x = cig_mod(theta)
    gauss_like = -0.5*np.dot(np.dot(x-dat,cinv),x-dat)
    return np.log(gauss_like)
def lnprior(theta):
    x,y = theta
    return 0.0
def lnprob(theta):
    lp = lnprior(theta)
    if lp == -np.inf:
        return -np.inf
    return lp + model(theta)

In [ ]:
def main(p0,nwalkers,niter,ndim,lnprob,data):
    sampler =  emcee.EnsembleSampler(nwalkers,ndim, lnprob, args = data)
    print('running baybee..')
    p0, _, _ = sampler.run_mcmc(p0,1000)
    sampler.reset()
    
    print( 'running prod')
    pos, prob, state =  sampler.run_mcmc(p0,niter)
    print('done!')
    return sampler, pos, prob, state

In [ ]:
data = (dat,cov,cinv)
nwalkers = 5
niter = 1000
# set n walkers
# set niter 
initial = np.array([2e-7,12.3]) # these come from the initial vierro params
ndim = 2 # only 2 params so 2 dim
p0 = [np.array(initial) + np.random.randn(ndim) for i in range(nwalkers)]

In [ ]:
sampler, pos, prob, state = main(p0,nwalkers,niter,ndim,lnlike,data)
samples = sampler.flatchain
labels = ['L_o', 'Log_Meff']
fig = corner.corner(samples,show_titles=True,labels=labels,plot_datapoints=True,quantiles=[0.16, 0.5, 0.84])